<a href="https://colab.research.google.com/github/weiru00/NLP_Assg2/blob/main/NLP_Assg2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Reading Training Dataset**

In [94]:
train_en = open("/content/drive/MyDrive/NLP_Assg2/data/data/train.en").read()
train_my = open("/content/drive/MyDrive/NLP_Assg2/data/data/train.my").read()
train_cl = open("/content/drive/MyDrive/NLP_Assg2/data/data/train.cl").read()

In [95]:
# Convert strings into list
train_en = list(train_en.splitlines())
train_my = list(train_my.splitlines())
train_cl = list(train_cl.splitlines())

**Tokenization**

In [96]:
import nltk
from nltk.tokenize import TweetTokenizer

In [97]:
tokenizer = TweetTokenizer()

In [98]:
train_en_tokens = [tokenizer.tokenize(s) for s in train_en]
train_my_tokens = [tokenizer.tokenize(s) for s in train_my]

In [99]:
from keras.preprocessing.text import Tokenizer

# Initialize the Tokenizer
tokenizer_k = Tokenizer()

In [100]:
# This section is to convert tokens to sequences to fit in the NN model later
# Fit the tokenizer on your training data
tokenizer_k.fit_on_texts(train_en_tokens + train_my_tokens) ##NOT USEFUL???

# Convert the tokens to sequences
train_en_sequences = tokenizer_k.texts_to_sequences(train_en_tokens)
train_my_sequences = tokenizer_k.texts_to_sequences(train_my_tokens)

In [101]:
max_length = max(map(len, train_en_sequences + train_my_sequences))
print(max_length)

548


In [102]:
from keras_preprocessing.sequence import pad_sequences

#  Pad the sequence to make them having the same length
train_en_sequences = pad_sequences(train_en_sequences, maxlen=max_length)
train_my_sequences = pad_sequences(train_my_sequences, maxlen=max_length)

In [103]:
len(train_my_sequences)

150000

In [104]:
train_my_sequences

array([[    0,     0,     0, ...,  2222,    37,  2434],
       [    0,     0,     0, ...,  2222,    37,  2434],
       [    0,     0,     0, ...,    17,  5363,    36],
       ...,
       [    0,     0,     0, ...,  1945,  5269,     1],
       [    0,     0,     0, ...,    13, 24349,     1],
       [    0,     0,     0, ...,    13, 24349,     1]], dtype=int32)

In [105]:
# train_df = pd.DataFrame({'train_en_tokens' : train_en_tokens,
#                          'train_my_tokens' : train_my_tokens,
#                           'train_cl' : train_cl,}, 
#                           columns=['train_en_tokens','train_my_tokens', 'train_cl'])

In [106]:
len(vocab_tokens)

300000

In [107]:
# combine two languages tokens
vocab_tokens = train_en_tokens + train_my_tokens

# get the vocab size for the combined lists
vocab_size = len(set(str(x) for x in vocab_tokens))
print(vocab_size)

180824


In [108]:
from gensim.models import Word2Vec

model = Word2Vec(vocab_tokens, min_count=1, size=300, window=5, sg=1, workers=4)

# train the model
model.train(vocab_tokens, total_examples=len(vocab_tokens), epochs=20)

(56328209, 68905600)

In [109]:
embedding_dim = model.wv.vector_size

In [110]:
print(embedding_dim)

300


Compare model trained and pretrained model

In [111]:
# from gensim.models import KeyedVectors
# model_compare = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/data/GoogleNews-vectors-negative300.bin', binary=True)

In [112]:
# print(model.wv.similarity('have', 'got'))

In [113]:
# print(model_compare.wv.similarity('have', 'got'))

Build NN model

In [114]:
from keras.layers import Input, LSTM, Embedding, Dot, Concatenate, Dense, Lambda
from keras.models import Model

In [115]:
# define the input layers for the two language data
input_en = Input(shape=(max_length,))
input_my = Input(shape=(max_length,))

In [116]:
# Define the embedding layer, with a fixed vocabulary size and embedding dimension
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_length)

# Apply the embedding layer to the input layers
embedded_en = embedding_layer(input_en)
embedded_my = embedding_layer(input_my)

In [117]:
# Define the LSTM cells for the Siamese RNN
lstm_layer = LSTM(64)

In [118]:
# Apply the LSTM cells to the embedded input layers
encoded_en = lstm_layer(embedded_en)
encoded_my = lstm_layer(embedded_my)

In [119]:
# Define the distance function
distance = Lambda(function=lambda x: abs(x[0] - x[1]), output_shape=lambda x: x[0])([encoded_en, encoded_my])

In [120]:
# Define the output layer
output = Dense(1, activation='sigmoid')(distance)

In [121]:
# Create the model
model_nn = Model(inputs=[input_en, input_my], outputs=output)
model_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [122]:
from keras.callbacks import ModelCheckpoint

checkpoint_path = "/content/drive/MyDrive/NLP_Assg2/Models/model_{epoch:02d}.h5"

# Create a ModelCheckpoint object
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, save_freq='epoch')

In [123]:
import numpy as np

In [124]:
model_nn.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 548)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 548)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 548, 300)     54247200    ['input_7[0][0]',                
                                                                  'input_8[0][0]']                
                                                                                                  
 lstm_2 (LSTM)                  (None, 64)           93440       ['embedding_2[0][0]',      

In [ ]:
# Train the model
model_nn.fit([train_en_sequences, train_my_sequences], np.array(train_cl).astype(int), epochs=5, batch_size=32)


Epoch 1/5
4688/4688 [==============================] - 8488s 2s/step - loss: 0.2499 - accuracy: 0.8965
Epoch 2/5
1231/4688 [======>.......................] - ETA: 1:43:37 - loss: 0.0991 - accuracy: 0.9660

In [ ]:
# print(np.shape(train_en_sequences))
# print(np.shape(train_my_sequences))
# print(np.shape(train_cl))

**Reading Validation Dataset**

In [ ]:
dev_en = open("/content/drive/MyDrive/NLP_Assg2/data/data/dev.en").read()
dev_my = open("/content/drive/MyDrive/NLP_Assg2/data/data/dev.my").read()
dev_cl = open("/content/drive/MyDrive/NLP_Assg2/data/data/dev.cl").read()

In [ ]:
# Convert strings into list
dev_en = list(train_en.splitlines())
dev_my = list(train_my.splitlines())
dev_cl = list(train_cl.splitlines())

In [ ]:
# model_nn.fit([train_en_sequences, train_my_sequences], np.array(train_cl).astype(int), epochs=5, batch_size=32, callbacks=[checkpoint])